# Projeto 1 - Ciência dos Dados

Nome: Luiz Ricardo Hardman Paranhos

Nome: Matheus Kwon

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [15]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [16]:
import os

filename = 'cheetos.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo cheetos.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [17]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  relevancia
0                            @eaipaixao_ cheetos obv         1.0
1  @giovannapinto mano eu fui comprar 2 cheetos e...         1.0
2                         @zabele__ só tenho cheetos         0.0
3  @lilizvante cheetos, old\nhttps://t.co/atamf2woj7         0.0
4  @biaventura14 kkkkkkkkkkkkkkkkkkkkk cheetos ve...         1.0

In [18]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  relevancia
0  sou do tipo que quando tá conversando com algu...           0
1  att fresquinha de flaming hot cheetos!!!! http...           0
2  me entupindo de cheetos enquanto vejo tapas e ...           0
3         cheetos ou doritos https://t.co/caytqjylnp           0
4     mas quem colocou o cheetos na minha tml hoje ?           0

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

- Relevante: informação da qual a empresa pode ganhar dinheiro
- Irrelevante: resto

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [19]:
train['relevancia'] = train['relevancia'].astype('category')
train.relevancia.cat.categories = ['irrelevante', 'relevante']

In [20]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/,_@¿]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def cleanup_enter(text):
    punctuation = '\n'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def link(text):
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\S+', '', text, flags=re.MULTILINE)
    return text

In [21]:
# String com todas as palavras
tweets_treinamento_raw = ''
for tweet in train.Treinamento:
    tweets_treinamento_raw = tweets_treinamento_raw + ' ' + tweet
tweets_treinamento = deEmojify(cleanup_enter(cleanup(link(tweets_treinamento_raw.lower()))))

In [22]:
# String com os relevantes
filtro_relevante_train = train.relevancia == 'relevante'
treinamento_relevante = train.loc[filtro_relevante_train,:]

tweets_relevantes = ''
for tweet in treinamento_relevante.Treinamento:
    tweets_relevantes = tweets_relevantes + ' ' + tweet

treinamento_relevante = deEmojify(cleanup_enter(cleanup(link(tweets_relevantes.lower()))))

In [23]:
# String com os irrelevantes
filtro_irrelevante_train = train.relevancia == 'irrelevante'
treinamento_irrelevante = train.loc[filtro_irrelevante_train,:]

tweets_irrelevantes = ''
for tweet in treinamento_irrelevante.Treinamento:
    tweets_irrelevantes = tweets_irrelevantes + ' ' + tweet

    
treinamento_irrelevante = deEmojify(cleanup_enter(cleanup(tweets_irrelevantes.lower())))

In [24]:
# Criando lista e séries a partir das strings
tweets_treinamento_lista = tweets_treinamento.split()
treinamento_relevante_lista = treinamento_relevante.split()
treinamento_irrelevante_lista = treinamento_irrelevante.split()

serie_treinamento = pd.Series(tweets_treinamento_lista)
serie_treinamento_relevante = pd.Series(treinamento_relevante_lista)
serie_treinamento_irrelevante = pd.Series(treinamento_irrelevante_lista)

In [25]:
tabela_treinamento = serie_treinamento.value_counts()
tabela_treinamento

cheetos    297
de         171
e          116
um          81
o           81
          ... 
stans        1
doritim      1
maya         1
humild       1
otários      1
Length: 1236, dtype: int64

In [26]:
tabela_treinamento_relevante = serie_treinamento_relevante.value_counts()
tabela_treinamento_relevante

cheetos    151
de          92
e           48
um          39
eu          31
          ... 
🤤            1
obv          1
meche        1
—            1
jantar       1
Length: 554, dtype: int64

In [45]:
stopwords = ['por', 'se','uma','em','mais','q','que','na','da','do','no','eu','de','e','o','um','a','com','pra', 'mas', 'só', 'so']

In [46]:
# Removendo stop words da lista de todas as palavras
tweets_treinamento_stopwords_lista = []
for palavra in tweets_treinamento_lista:
    x = 0
    if palavra in stopwords:
        x = 1
    if x == 0:
        if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
            tweets_treinamento_stopwords_lista.append(palavra[:-3]+'r')
        elif palavra == 'n' or palavra == 'nao':
            tweets_treinamento_stopwords_lista.append('não')
        else:
            tweets_treinamento_stopwords_lista.append(palavra)
        

serie_treinamento_filtrado = pd.Series(tweets_treinamento_stopwords_lista)
tabela_treinamento_filtrado = serie_treinamento_filtrado.value_counts(True)
tabela_treinamento_filtrado.head(20)

cheetos      0.114407
é            0.023112
não          0.019260
comer        0.014638
requeijão    0.011941
doritos      0.010015
tem          0.010015
meu          0.008860
me           0.007319
gosto        0.005393
pq           0.005008
melhor       0.005008
minha        0.004622
vontade      0.004622
cheiro       0.004622
vou          0.004622
como         0.004237
queijo       0.004237
pacote       0.003852
fandangos    0.003852
dtype: float64

In [47]:
# Removendo stop words da lista de todas as palavras relevantes
treinamento_relevante_stopwords_lista = []
for palavra in treinamento_relevante_lista:
    x = 0
    if palavra in stopwords:
        x = 1
    if x == 0:
        if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
            treinamento_relevante_stopwords_lista.append(palavra[:-3]+'r')
        elif palavra == 'n' or palavra == 'nao':
            treinamento_relevante_stopwords_lista.append('não')
        else:
            treinamento_relevante_stopwords_lista.append(palavra)

serie_treinamento_relevante_filtrado = pd.Series(treinamento_relevante_stopwords_lista)
tabela_treinamento_relevante_filtrado = serie_treinamento_relevante_filtrado.value_counts(True)
tabela_treinamento_relevante_filtrado.head(20)

cheetos       0.140204
é             0.027855
requeijão     0.025998
comer         0.022284
doritos       0.017642
gosto         0.011142
meu           0.010214
melhor        0.010214
vontade       0.010214
queria        0.009285
não           0.009285
tem           0.008357
fandangos     0.007428
me            0.007428
ou            0.006500
sabor         0.006500
pacote        0.006500
pq            0.006500
salgadinho    0.006500
bom           0.005571
dtype: float64

In [55]:
# Removendo stop words da lista de todas as palavras irrelevantes
treinamento_irrelevante_stopwords_lista = []
for palavra in treinamento_irrelevante_lista:
    x = 0
    if palavra in stopwords:
        x = 1
    if x == 0:
        if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
            treinamento_irrelevante_stopwords_lista.append(palavra[:-3]+'r')
        elif palavra == 'n' or palavra == 'nao':
            treinamento_irrelevante_stopwords_lista.append('não')
        else:
            treinamento_irrelevante_stopwords_lista.append(palavra)

serie_treinamento_irrelevante_filtrado = pd.Series(treinamento_irrelevante_stopwords_lista)
tabela_treinamento_irrelevante_filtrado = serie_treinamento_irrelevante_filtrado.value_counts(True)
tabela_treinamento_irrelevante_filtrado.head(30)

cheetos     0.087139
não         0.024038
é           0.018029
tem         0.010216
comer       0.008413
meu         0.007212
me          0.006611
cheiro      0.006010
vou         0.006010
ele         0.005409
como        0.004808
minha       0.004808
seu         0.004207
tipo        0.004207
aqui        0.004207
as          0.003606
doritos     0.003606
vc          0.003606
pq          0.003606
fazer       0.003606
comprar     0.003005
queijo      0.003005
os          0.003005
vai         0.003005
parece      0.003005
isso        0.003005
pelo        0.003005
estar       0.003005
biscoito    0.003005
essa        0.003005
dtype: float64

In [49]:
P_R = len(treinamento_relevante_stopwords_lista) / len(tweets_treinamento_stopwords_lista)
print('P(R):',P_R)
P_Rc = 1 - P_R
print('P(Rc):',P_Rc)
print(P_R + P_Rc)

P_RequeijaoDadoR = tabela_treinamento_relevante_filtrado['requeijão']
P_Requeijao = tabela_treinamento_filtrado['requeijão']
P_RDadoRequeijao = P_RequeijaoDadoR * P_R / P_Requeijao
print('P(R|Requeijao):',P_RDadoRequeijao)

P_RequeijaoDadoRc = tabela_treinamento_irrelevante_filtrado['requeijão']
P_RcDadoRequeijao = P_RequeijaoDadoRc * P_Rc / P_Requeijao
print('P(Rc|Requeijao):',P_RcDadoRequeijao)

# print(P_RequeijaoDadoRc)
# print(P_RequeijaoDadoR)
print(P_RDadoRequeijao+P_RcDadoRequeijao)


P(R): 0.41486902927580893
P(Rc): 0.5851309707241911
1.0
P(R|Requeijao): 0.9032258064516128
P(Rc|Requeijao): 0.08834134615384615
0.9915671526054589


In [50]:
P_DoritosDadoR = tabela_treinamento_relevante_filtrado['doritos']
P_Doritos = tabela_treinamento_filtrado['doritos']
P_RDadoDoritos = P_DoritosDadoR * P_R / P_Doritos
print('P(R|Doritos):',P_RDadoDoritos)

P_DoritosDadoRc = tabela_treinamento_irrelevante_filtrado['doritos']
P_RcDadoDoritos = P_DoritosDadoRc * P_Rc / P_Doritos
print('P(Rc|Doritos):',P_RcDadoDoritos)

# print(P_DoritosDadoRc)
# print(P_DoritosDadoR)
print(P_RDadoDoritos+P_RcDadoDoritos)


P(R|Doritos): 0.7307692307692307
P(Rc|Doritos): 0.21066013313609466
0.9414293639053254


In [51]:
P_ComerDadoR = tabela_treinamento_relevante_filtrado['comer']
P_Comer = tabela_treinamento_filtrado['comer']
P_RDadoComer = P_ComerDadoR * P_R / P_Comer
print('P(R|Comer):',P_RDadoComer)

P_ComerDadoRc = tabela_treinamento_irrelevante_filtrado['comer']
P_RcDadoComer = P_ComerDadoRc * P_Rc / P_Comer
print('P(Rc|Comer):',P_RcDadoComer)

# print(P_ComerDadoRc)
# print(P_ComerDadoR)
print(P_RDadoComer+P_RcDadoComer)


P(R|Comer): 0.6315789473684211
P(Rc|Comer): 0.3363170546558705
0.9678960020242916


In [52]:
P_CheiroDadoR = tabela_treinamento_relevante_filtrado['cheiro']
P_Cheiro = tabela_treinamento_filtrado['cheiro']
P_RDadoCheiro = P_CheiroDadoR * P_R / P_Cheiro
print('P(R|Cheiro):',P_RDadoCheiro)

P_CheiroDadoRc = tabela_treinamento_irrelevante_filtrado['cheiro']
P_RcDadoCheiro = P_CheiroDadoRc * P_Rc / P_Cheiro
print('P(Rc|Cheiro):',P_RcDadoCheiro)

# print(P_CheiroDadoRc)
# print(P_CheiroDadoR)
print(P_RDadoCheiro+P_RcDadoCheiro)


P(R|Cheiro): 0.16666666666666666
P(Rc|Cheiro): 0.7607171474358974
0.927383814102564


In [53]:
P_SalgadinhoDadoR = tabela_treinamento_relevante_filtrado['salgadinho']
P_Salgadinho = tabela_treinamento_filtrado['salgadinho']
P_RDadoSalgadinho = P_SalgadinhoDadoR * P_R / P_Salgadinho
print('P(R|Salgadinho):',P_RDadoSalgadinho)

P_SalgadinhoDadoRc = tabela_treinamento_irrelevante_filtrado['salgadinho']
P_RcDadoSalgadinho = P_SalgadinhoDadoRc * P_Rc / P_Salgadinho
print('P(Rc|Salgadinho):',P_RcDadoSalgadinho)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadoSalgadinho+P_RcDadoSalgadinho)


P(R|Salgadinho): 0.7777777777777777
P(Rc|Salgadinho): 0.20285790598290598
0.9806356837606837


In [54]:
P_naoDadoR = tabela_treinamento_relevante_filtrado['não']
P_nao = tabela_treinamento_filtrado['não']
P_RDadonao = P_naoDadoR * P_R / P_nao
print('P(R|nao):',P_RDadonao)

P_naoDadoRc = tabela_treinamento_irrelevante_filtrado['não']
P_RcDadonao = P_naoDadoRc * P_Rc / P_nao
print('P(Rc|nao):',P_RcDadonao)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadonao+P_RcDadonao)

P(R|nao): 0.19999999999999998
P(Rc|nao): 0.7302884615384616
0.9302884615384616


In [56]:
P_biscoitoDadoR = tabela_treinamento_relevante_filtrado['biscoito']
P_biscoito = tabela_treinamento_filtrado['biscoito']
P_RDadobiscoito = P_biscoitoDadoR * P_R / P_biscoito
print('P(R|biscoito):',P_RDadobiscoito)

P_biscoitoDadoRc = tabela_treinamento_irrelevante_filtrado['biscoito']
P_RcDadobiscoito = P_biscoitoDadoRc * P_Rc / P_biscoito
print('P(Rc|biscoito):',P_RcDadobiscoito)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadobiscoito+P_RcDadobiscoito)

P(R|biscoito): 0.2857142857142857
P(Rc|biscoito): 0.6520432692307692
0.9377575549450549


In [57]:
P_comprarDadoR = tabela_treinamento_relevante_filtrado['comprar']
P_comprar = tabela_treinamento_filtrado['comprar']
P_RDadocomprar = P_comprarDadoR * P_R / P_comprar
print('P(R|comprar):',P_RDadocomprar)

P_comprarDadoRc = tabela_treinamento_irrelevante_filtrado['comprar']
P_RcDadocomprar = P_comprarDadoRc * P_Rc / P_comprar
print('P(Rc|comprar):',P_RcDadocomprar)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadocomprar+P_RcDadocomprar)

P(R|comprar): 0.4444444444444445
P(Rc|comprar): 0.5071447649572649
0.9515892094017093


In [58]:
P_queijoDadoR = tabela_treinamento_relevante_filtrado['queijo']
P_queijo = tabela_treinamento_filtrado['queijo']
P_RDadoqueijo = P_queijoDadoR * P_R / P_queijo
print('P(R|queijo):',P_RDadoqueijo)

P_queijoDadoRc = tabela_treinamento_irrelevante_filtrado['queijo']
P_RcDadoqueijo = P_queijoDadoRc * P_Rc / P_queijo
print('P(Rc|queijo):',P_RcDadoqueijo)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadoqueijo+P_RcDadoqueijo)

P(R|queijo): 0.5454545454545455
P(Rc|queijo): 0.4149366258741259
0.9603911713286715


In [60]:
P_melhorDadoR = tabela_treinamento_relevante_filtrado['melhor']
P_melhor = tabela_treinamento_filtrado['melhor']
P_RDadomelhor = P_melhorDadoR * P_R / P_melhor
print('P(R|melhor):',P_RDadomelhor)

P_melhorDadoRc = tabela_treinamento_irrelevante_filtrado['melhor']
P_RcDadomelhor = P_melhorDadoRc * P_Rc / P_melhor
print('P(Rc|melhor):',P_RcDadomelhor)

# print(P_SalgadinhoDadoRc)
# print(P_SalgadinhoDadoR)
print(P_RDadomelhor+P_RcDadomelhor)

P(R|melhor): 0.8461538461538461
P(Rc|melhor): 0.14044008875739647
0.9865939349112426


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [ ]:
string = 'comendo'
print(string[:-4])